In [1]:
import os
import glob
import duckdb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [ ]:
# data = pd.read_csv('combined_data.csv')
# data.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,apps,churn
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.000,Prepaid,106.740,139.720,47.310,59.450,0,NaN,25,18.000,13,1.340,[],False
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.000,Prepaid,31.550,12.140,21.520,1221.650,0,NaN,51,8.000,3,2.570,[],False
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.000,Postpaid,30.640,10.170,31.110,1170.450,0,1.000,0,13.000,10,8.020,[],False
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.000,Prepaid,NaN,NaN,36.030,2418.910,0,NaN,19,7.000,11,5.960,['RitimGo'],False
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.000,Prepaid,85.620,164.790,46.400,1518.190,0,NaN,99,15.000,6,8.290,[],False


Accuracy : hesaplamasak da olur.

Precision : True Positive Rate => TP / (TP + FP)   (+ dediklerimin ne kadarı gerçekten öyle , modelin + işaretleri ne kadar güvenilir ? False Positive endişem var)

-->--> Örneğin, 0.78 precision, modelin churn tahminlerinin %78'inin doğru olduğunu gösterir.


Recall : Gerçekte + olanların ne kadarını + olarak işaretleyebiliyor  => (TP / TP + FN )  [FAlse Negative en büyük endişem ise , bu arttıkça yanlış alarmların oranı azalır]

-->--> Örneğin, 0.65 recall, modelin gerçek churn'lerin %65'ini yakaladığını gösterir.

Gerçek churn'lerin ne kadarını yakaladığını gösterir. Özellikle churn'leri kaçırmak istemiyorsan, recall önemlidir.


False Positive Rate : Yanlış alarm => FP / (TP + FN) Az olsun istiyorum.


id: Müşteri IDsi

Silinmeli zaten.


age: Müşterinin yaşı

    yaşlar genel olarak azalan şeklide ilerliyor , en çok müşteri 16-20 yaş arasında sonra 20-24 ve o şekilde sıralı bir şekilde gidiyor.
    min - 18 
    max - 80
    sola çarpık bir veri.

    kategorik hale getirilebilir , ortalama yanıltıcıdır , median dikkate alınabilir ama müşterilerin çoğunun genç sayılabileceği unutulmamalı.




tenure: Müşterinin operatörde geçirdiği toplam süre (ay cinsinden)

    280 aya kadar süre geçirren 8 milyon 400K kişi var yaklaşık olarak. Ondan sonra dikkate değer bir şekilde bu sayı düşüyor.

    Bu değerler yıla dönüştürülebilir , ve kategorik hale getirilebilir yine.

    Bu sütunlar farklı özellikler arasında bir ilişki olup olmadığı mutlaka incelenmelidir!!



service_type: Ön Ödemeli, Peşin Ödemeli veya Geniş Bant internet müşterisi

Prepaid : Ön ödemeli bağlantı, kullanıcıların kullandıkları telekom hizmetleri için peşin ödeme yapmak zorunda oldukları bir hücresel bağlantıdır

Postpaid : Faturalı bağlantı, kullanıcıların önce telekom hizmetlerini kullandığı ve daha sonra aylık fatura döngüsüne göre ödeme yaptığı bağlantıdır.

Geniş Bant internet müşterisi : Sadece ev interneti almış , yani call_drop olma şansı yok zaten..

Burada bir gruplama yapıp call_drop işini halletmek lazım ayrıca her grubun tenure oranını da incelemek gerekiyor.

Burada eksik veri yok ! tüm alt kategoriler neredeyse eşit dağılmış durumda.

Bunlar arasında faturalı ve faturasızların yaş ortalamalarına bak , ne kadar konuştuklarına bak.


avg_call_duration: Ortalama sesli görüşme süresi (saniye)

    * bence bu sütun önemli 

data_usage: GB Upload + Download -> Aylık Ortalama
    servis tiğine göre data_usage değişimi önemli bir durum bence.

    Burada histogram oluşturunca dağılım düz geliyor.
    Eksik veri sayısı çok değil ama varbu arada.

roaming_usage:
     Ortalama roaming sesli görüşme süresi Roaming, bağlı olduğunuz operatörün hizmet sağlama sahasından çıktığınızda farklı bir operatörün sağladığı kapsama saha hizmetini alabilmeniz anlamına geliyor. 

     eksik veri sayısı üstteki gibi fazla değil ama var.

    

monthly_charge: Aylık ortalama fatura.
    en çok incelenmesi gereken özellik.

    Yaş gruplarının faturaları nasıl değişiyor ? 

    200-400 arasında bir yığılma var .
    sola çarpık veri söz konusu , ama sağa doğru biraz fazla uzanıor muhtemelen bazı müşteriler kurumsal firma alan 2K tl üstü fatura ödeyenler kişisel kullanıyor olamaz bence.



overdue_payments: Ödemesi geçen fatura adedi
Büyük Çoğunluk 0 da yoğunlaşmış durumda. 4.5 milyon kadar.

Kalan ödeme gecikme sayısı ise eşit dağılıyor gibi 1,2,3,4,5 olma ihtimali hemen hemen aynı hepsi 1.1 milyon kişi.


auto_payment: Otomatik ödeme talimatı
    Bu da önemli bir flag ! otomatik ödeme yapan adama sürekli ilgilenmiyordur , kalıcı sayılabilir bi incelemek lazım.
    % 33 lük bir veri kaybı var bu sütunda.



avg_top_up_count: 
    Ön yüklemeli abone için aylık yükleme sayısı ortalaması ????
    ayda kaç defa kontör yüklemiş , son 6 ayın ortalaması !!! veya 3 ayın ortalaması.
    
    

call_drops: Şebekede yaşadığı sesli görüşme kesilmesi.
    Bu durum fazla yaşanmasına rağmen churn durumu olmayan insanlar kesinlikle ayrılmaz da diyebilirim bence.
    Hiç drop olmamasına rağmen ayrılıyorsa muhtemelen bizden uzak olsun sınıfı sayılır belki de.
Eksik veri oranı fazla sayılır : %33




satisfaction_score:

     Müşteri çağrı merkezi değerlendirme skoru
     missing yok
     

apps: 
    Müşterinin kullandığı diğer servislerimiz İzleGo -> 1 RitimGo -> 2 CüzdanX -> 3 HızlıPazar -> 4 Konuşalım -> 5

    Missing: % 0
    [] : 68%
    ['Konuşalım'] : 17%
    ['RitimGo'] : 8%
    Other : 7%

Feature Engineering Notları :

df["age_group"] = pd.cut(df["age"], bins=[18, 30, 45, 60, 80], labels=["young", "adult", "middle_aged", "senior"])

-------------------------------

df["tenure_years"] = df["tenure"] // 12  # Ayları yıllara çevir
df["tenure_category"] = pd.cut(df["tenure_years"], bins=[0, 1, 3, 5, 10, 20], labels=["0-1", "1-3", "3-5", "5-10", "10+"])

--------------------------------------------

df["monthly_charge_group"] = pd.cut(df["monthly_charge"], bins=[0, 50, 100, 200, 500], labels=["low", "medium", "high", "very_high"])

---------------------------------------------

df["call_duration_per_month"] = df["avg_call_duration"] / (df["tenure"] + 1)  # Bölme hatasını önlemek için +1

----------------------------------------------
Otomatik ödeme kullanmayan ancak çok fazla ödeme gecikmesi yaşayanları belirleyebiliriz.

python
Kopyala
Düzenle


df["high_risk_payment"] = ((df["auto_payment"] == False) & (df["overdue_payments"] > 2)).astype(int)

---------------------------------------------
Kullanıcı Ne Kadar Servis Kullanıyor? (Apps Sayısı)

df["num_apps_used"] = df["apps"].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)  # Eğer string olarak geldiyse listeye çevirip say


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,10000000.000,30.376,11.400,18.000,20.000,28.000,38.000,80.000
tenure,9975200.000,158.252,94.755,1.000,78.000,156.000,234.000,754.000
avg_call_duration,6335687.000,60.512,34.343,1.000,30.770,60.500,90.270,120.000
data_usage,9500467.000,100.066,57.703,0.100,50.100,100.050,150.060,200.000
roaming_usage,6668788.000,29.991,17.320,0.000,14.980,30.000,44.990,60.000
monthly_charge,9749929.000,743.768,597.310,30.000,317.760,480.610,1018.330,2500.000
overdue_payments,10000000.000,1.666,1.826,0.000,0.000,1.000,3.000,5.000
auto_payment,6655252.000,0.500,0.500,0.000,0.000,0.000,1.000,1.000
avg_top_up_count,10000000.000,17.520,29.484,0.000,0.000,0.000,29.000,100.000
call_drops,6668788.000,10.003,6.056,0.000,5.000,10.000,15.000,20.000


In [2]:
from sklearn.model_selection import train_test_split


In [9]:
# %10'unu test seti olarak ayır
# train_df, test_df = train_test_split(data, test_size=0.1, stratify=data["churn"], random_state=42)

# Test setini kaydet
# test_df.to_csv("test_set.csv", index=False)

# Train set ile devam edelim
# train_df.to_csv("train_set.csv", index=False)
# print("Test seti kaydedildi, train set ile devam ediyoruz.")

Test seti kaydedildi, train set ile devam ediyoruz.


stratify parametresi : 
imbalanced (dengesiz) datasetlerde sınıf dağılımının train ve test setlerinde orijinal veriyle aynı oranda kalmasını sağlar. Bu özellikle azınlık sınıfının (örneğin fraud/churn vakaları) çok az olduğu durumlarda kritik öneme sahiptir.

Sınıf Oranını Korur: Orijinal verideki fraud/non-fraud veya churn/non-churn oranı, train ve test setlerinde de aynı kalır.

Modelin Yanlılığını Önler: Rastgele bölme işlemi sırasında azınlık sınıfının tamamen bir tarafa kaymasını engeller.

Daha Gerçekçi Performans: Test seti, gerçek dünyadaki dağılımı yansıtır (örneğin %1 fraud varsa, testte de ~%1 fraud olur).


In [12]:
def feature_engineering(df):
    """Feature Engineering işlemlerini uygular"""

    # Age Gruplama
    df["age_group"] = pd.cut(df["age"], bins=[18, 30, 45, 60, 80], labels=["young", "adult", "middle_aged", "senior"])

    # Tenure Yıla Çevirme ve Kategorik Yapma
    df["tenure_years"] = df["tenure"] // 12
    df["tenure_category"] = pd.cut(df["tenure_years"], bins=[0, 1, 3, 5, 10, 20], labels=["0-1", "1-3", "3-5", "5-10", "10+"])

    # Monthly Charge Kategorik Hale Getirme
    df["monthly_charge_group"] = pd.cut(df["monthly_charge"], bins=[0, 50, 100, 200, 500], labels=["low", "medium", "high", "very_high"])

    # Ortalama Konuşma Süresi / Tenure Oranı
    df["call_duration_per_month"] = df["avg_call_duration"] / (df["tenure"] + 1)

    # Ödeme Risk Skoru (Auto Payment & Geciken Ödemeler)
    df["high_risk_payment"] = ((df["auto_payment"] == False) & (df["overdue_payments"] > 2)).astype(int)

    # Kullanıcı Kaç Servis Kullanıyor? (Apps Sayısı)
    df["num_apps_used"] = df["apps"].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)

    return df


In [6]:
df = pd.read_csv('train_set.csv')
df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,apps,churn
0,bf129dc3-7010-48bc-8946-574866665e0f,18,27.000,Prepaid,5.370,67.030,24.780,414.760,0,NaN,35,12.000,0,7.500,[],False
1,c970b11d-1402-4c21-83ff-0a3a6b1edd68,18,42.000,Prepaid,71.250,66.980,2.030,2429.210,0,NaN,87,7.000,3,2.530,['Konuşalım'],False
2,30592b42-01e7-4cfb-9ca5-df296f36bbfb,51,185.000,Broadband,NaN,174.190,NaN,460.050,2,1.000,0,NaN,5,1.910,['RitimGo'],False
3,3a5f2d11-8b3b-4d25-9df4-cfff6406a7e1,32,158.000,Postpaid,106.270,60.480,14.300,620.030,2,0.000,0,8.000,6,8.690,['Konuşalım'],False
4,4c27fc61-bb70-45e0-80f7-bc5339be2980,50,80.000,Postpaid,73.270,2.380,15.870,613.530,2,0.000,0,6.000,10,5.520,[],False


In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,9000000.000,30.376,11.399,18.000,20.000,28.000,38.000,80.000
tenure,8977721.000,158.250,94.753,1.000,78.000,156.000,234.000,754.000
avg_call_duration,5702402.000,60.511,34.343,1.000,30.760,60.500,90.270,120.000
data_usage,8550345.000,100.065,57.696,0.100,50.120,100.050,150.050,200.000
roaming_usage,6002225.000,29.993,17.319,0.000,14.980,30.000,44.990,60.000
monthly_charge,8774838.000,743.832,597.345,30.000,317.800,480.630,1018.370,2500.000
overdue_payments,9000000.000,1.666,1.826,0.000,0.000,1.000,3.000,5.000
auto_payment,5989294.000,0.500,0.500,0.000,0.000,0.000,1.000,1.000
avg_top_up_count,9000000.000,17.522,29.483,0.000,0.000,0.000,29.000,100.000
call_drops,6002225.000,10.004,6.056,0.000,5.000,10.000,15.000,20.000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000000 entries, 0 to 8999999
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      object 
 1   age                     int64  
 2   tenure                  float64
 3   service_type            object 
 4   avg_call_duration       float64
 5   data_usage              float64
 6   roaming_usage           float64
 7   monthly_charge          float64
 8   overdue_payments        int64  
 9   auto_payment            float64
 10  avg_top_up_count        int64  
 11  call_drops              float64
 12  customer_support_calls  int64  
 13  satisfaction_score      float64
 14  apps                    object 
 15  churn                   bool   
dtypes: bool(1), float64(8), int64(4), object(3)
memory usage: 1.0+ GB


## Undersampling


Eğer RandomUnderSampler Denemek istersen aşağıdaki kodu kullan !

In [8]:
# from imblearn.under_sampling import RandomUnderSampler

# # RandomUnderSampler nesnesi oluştur
# undersample = RandomUnderSampler(sampling_strategy=0.1, random_state=42)

# # X ve y'yi yeniden dengele
# X_resampled, y_resampled = undersample.fit_resample(df.drop(columns=[target, "id"]), df[target])

# print("RandomUnderSampler tamamlandı! Yeni veri seti boyutları:")
# print(pd.Series(y_resampled).value_counts())


In [7]:
# Hedef değişken
target = "churn"

# Churn olanları ve olmayanları ayır
df_churn = df[df[target] == 1]  # Churn (1) olanlar
df_no_churn = df[df[target] == 0]  # Churn (0) olanlar

# No churn olanlardan churn'un 5 katı kadar örnek alalım (Random değil!)
df_no_churn_sampled = df_no_churn.sample(n=len(df_churn) * 5, random_state=42)

# Yeni veri setini oluştur
df_balanced = pd.concat([df_churn, df_no_churn_sampled])

# Karıştıralım
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("Undersampling tamamlandı! Yeni veri seti boyutları:")
print(df_balanced[target].value_counts())


Undersampling tamamlandı! Yeni veri seti boyutları:
churn
False    601440
True     120288
Name: count, dtype: int64


In [ ]:
df_balanced.iloc[1]

id                        838880ca-c300-4b85-adec-f5152a92cf4b
age                                                         19
tenure                                                 250.000
service_type                                           Prepaid
avg_call_duration                                       64.710
data_usage                                              98.600
roaming_usage                                            7.850
monthly_charge                                         594.170
overdue_payments                                             0
auto_payment                                               NaN
avg_top_up_count                                            38
call_drops                                              17.000
customer_support_calls                                      11
satisfaction_score                                       3.250
apps                                             ['Konuşalım']
churn                                                  

In [14]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721728 entries, 0 to 721727
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      721728 non-null  object 
 1   age                     721728 non-null  int64  
 2   tenure                  719923 non-null  float64
 3   service_type            721728 non-null  object 
 4   avg_call_duration       484890 non-null  float64
 5   data_usage              685435 non-null  float64
 6   roaming_usage           510460 non-null  float64
 7   monthly_charge          703791 non-null  float64
 8   overdue_payments        721728 non-null  int64  
 9   auto_payment            464854 non-null  float64
 10  avg_top_up_count        721728 non-null  int64  
 11  call_drops              510460 non-null  float64
 12  customer_support_calls  721728 non-null  int64  
 13  satisfaction_score      721728 non-null  float64
 14  apps                

In [15]:
# ID'yi düşür
df_balanced = df_balanced.drop(columns=["id"])

# Eksik değerleri median ile doldur
num_cols = ["tenure", "avg_call_duration", "data_usage", "roaming_usage", "monthly_charge", "call_drops"]
for col in num_cols:
    df_balanced[col] = df_balanced[col].fillna(df_balanced[col].median())

# Auto Payment NaN değerlerini 0 ile doldur (Otomatik ödeme yok anlamına gelir)
df_balanced["auto_payment"] = df_balanced["auto_payment"].fillna(0).astype(int)

print("Eksik değerler dolduruldu!")

Eksik değerler dolduruldu!


In [ ]:
# Service Type One-Hot Encoding
df_balanced = pd.get_dummies(df_balanced, columns=["service_type"], drop_first=True)

# Apps Kullanım Sayısını Hesapla
df_balanced["num_apps_used"] = df_balanced["apps"].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)

# Apps sütununu artık silebiliriz
df_balanced = df_balanced.drop(columns=["apps"])

print("Kategorik değişkenler sayısallaştırıldı!")

# bool olanları da sayısal yap
df_balanced[['service_type_Postpaid','service_type_Prepaid ', 'service_type_Prepaid']] = df_balanced[['service_type_Postpaid','service_type_Prepaid ', 'service_type_Prepaid']].astype(int)

In [20]:
print(df_balanced.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721728 entries, 0 to 721727
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     721728 non-null  int64  
 1   tenure                  721728 non-null  float64
 2   avg_call_duration       721728 non-null  float64
 3   data_usage              721728 non-null  float64
 4   roaming_usage           721728 non-null  float64
 5   monthly_charge          721728 non-null  float64
 6   overdue_payments        721728 non-null  int64  
 7   auto_payment            721728 non-null  int32  
 8   avg_top_up_count        721728 non-null  int64  
 9   call_drops              721728 non-null  float64
 10  customer_support_calls  721728 non-null  int64  
 11  satisfaction_score      721728 non-null  float64
 12  churn                   721728 non-null  bool   
 13  service_type_Postpaid   721728 non-null  bool   
 14  service_type_Prepaid

## Model Constructing


In [21]:
from sklearn.model_selection import train_test_split

# Hedef değişken
target = "churn"

# X ve y'yi ayır
X = df_balanced.drop(columns=[target])
y = df_balanced[target]

# Train-Validation ayır
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train ve Validation setleri oluşturuldu!")


Train ve Validation setleri oluşturuldu!


In [25]:
import lightgbm as lgb
from sklearn.metrics import classification_report, roc_auc_score

# LightGBM Modeli
model = lgb.LGBMClassifier(
    class_weight="balanced",  # Dengesiz veri için otomatik ağırlıklandırma
    random_state=42,
    n_estimators=200,
    learning_rate=0.05
)

# Modeli eğit
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],  # Validation verisi
    eval_metric="auc",  # Değerlendirme metriği
    callbacks=[lgb.early_stopping(20)]  # Doğru şekilde early stopping ekleme
)

print("Model eğitildi!")


[LightGBM] [Info] Number of positive: 96230, number of negative: 481152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,021881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1749
[LightGBM] [Info] Number of data points in the train set: 577382, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,500000 -> initscore=-0,000000
[LightGBM] [Info] Start training from score -0,000000
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[108]	valid_0's auc: 0.799717	valid_0's binary_logloss: 0.540714
Model eğitildi!


In [26]:
from sklearn.metrics import classification_report, roc_auc_score

# Tahmin yap
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1]  # AUC için ihtiyacımız olan olasılıklar

# Skorları hesapla
auc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC Score: {auc:.4f}")
print(classification_report(y_val, y_pred))

AUC Score: 0.7997
              precision    recall  f1-score   support

       False       0.95      0.67      0.78    120288
        True       0.33      0.81      0.47     24058

    accuracy                           0.69    144346
   macro avg       0.64      0.74      0.63    144346
weighted avg       0.84      0.69      0.73    144346

